In [1]:
library(tidyverse)
library(hash)
library(ggplot2)
library(org.Hs.eg.db)
library(clusterProfiler)
library(enrichplot)
library(ReactomePA)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
hash-2.2.6.2 provided by Decision Patterns


Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The

In [2]:
# Set random seed so this part is reproducible
# https://www.random.org/ 2023-08-09
set.seed(3866) 

In [3]:
tpm <- read.table("../../../../data/expression/processed/train//V4//IO-Atlas-NSCLC-TPM-TRAIN-2023-08-10-V4.tsv", 
                  sep='\t', 
                  row.names=1, 
                  header=T)

universe <- unlist(mget(row.names(tpm), envir=org.Hs.egENSEMBL2EG,
                       ifnotfound = NA))

universe <- as.vector(unique(na.omit(universe)))

In [4]:
length(universe)

[1] 16270

In [6]:
ratios <- read.table("../../../../data/ratios//IO-Atlas-NSCLC-NSCLC-Response-Cluster-2-TPM-MinMaxNorm-TRAIN-2023-08-10-V4-ratios.tsv",
                     sep='\t', 
                     header=T)

ratios <- separate_wider_delim(ratios, cols="X", delim=":::", names=c("numerator", "denominator"), cols_remove=F)

In [7]:
genes <- list()

for (c in unique(unlist(as.list(ratios["cluster"]))))
     {    
        up <- ratios %>% filter((cluster == c)) %>% pull("numerator") %>% unique() 
    
        up_eg <- unlist(
            mget(up , 
             envir=org.Hs.egENSEMBL2EG,
             ifnotfound = NA)
        )
    
        up_eg <- as.vector(unique(na.omit(up_eg)))
    
        genes[[as.character(sprintf("%s up", c))]] <- up_eg
    
        dwn <- ratios %>% filter((cluster == c)) %>% pull("denominator") %>% unique() 
    
        dwn_eg <- unlist(
            mget(dwn , 
             envir=org.Hs.egENSEMBL2EG,
             ifnotfound = NA)
        )
    
        dwn_eg <- as.vector(unique(na.omit(dwn_eg)))
    
        genes[[as.character(sprintf("%s dwn", c))]] <- dwn_eg
     }

In [31]:
enrich <- list()

for (key in names(genes)){
    
    print(key)
    
    enrich[[key]] <- list()
    
    #
    # GO Analysis
    #
    enrich.go <- enrichGO(gene = genes[[key]], 
                   ont = "BP",
                   OrgDb ="org.Hs.eg.db",
                   universe = universe,
                   readable=TRUE,
                   pAdjustMethod="fdr",
                   qvalueCutoff = 0.1)
    
    enrich.go <- as.data.frame(enrich.go)
    enrich[[key]][["GO"]] <- enrich.go
    
    #
    # KEGG Analysis
    #
    
    enrich.kegg <- enrichKEGG(gene = genes[[key]], 
                               organism='hsa',
                               pAdjustMethod="fdr",
                               universe = universe,
                               qvalueCutoff = 0.1)
    
    enrich.kegg <- as.data.frame(enrich.kegg)
    
    enrich[[key]][["KEGG"]] <- enrich.kegg
    
    }

[1] "ratio-cluster1 up"
[1] "ratio-cluster1 dwn"
[1] "ratio-cluster0 up"
[1] "ratio-cluster0 dwn"
[1] "ratio-cluster2 up"
[1] "ratio-cluster2 dwn"


In [32]:
genes[["ratio-cluster1 dwn"]]

[1] "5188"      "22859"     "147948"    "7260"      "10908"     "54896"    
  [7] "1497"      "55248"     "8578"      "9817"      "6604"      "55659"    
 [13] "94005"     "56970"     "410"       "79734"     "8846"      "1477"     
 [19] "8760"      "3420"      "9663"      "8555"      "65057"     "55756"    
 [25] "55011"     "56006"     "26024"     "8985"      "20"        "51003"    
 [31] "10014"     "54868"     "51296"     "11108"     "6539"      "8078"     
 [37] "7942"      "1416"      "4968"      "10289"     "9422"      "55147"    
 [43] "8237"      "1203"      "84695"     "66008"     "3065"      "10459"    
 [49] "54936"     "57539"     "23093"     "84078"     "23326"     "79637"    
 [55] "23263"     "81537"     "1678"      "51337"     "55695"     "157922"   
 [61] "7770"      "26061"     "84231"     "25974"     "57498"     "6927"     
 [67] "54806"     "55727"     "23143"     "8803"      "84085"     "51204"    
 [73] "2995"      "7094"      "11258"     "26235"     "580"       "6602"     
 [79] "54332"     "25842"     "7372"      "2647"      "9061"      "57192"    
 [85] "93145"     "121260"    "8816"      "55334"     "2588"      "63935"    
 [91] "3607"      "5546"      "57198"     "80775"     "9686"      "9125"     
 [97] "4794"      "144193"    "116843"    "10244"     "57563"     "6835"     
[103] "196743"    "54805"     "85458"     "54995"     "381"       "54947"    
[109] "78990"     "8467"      "221895"    "5550"      "51530"     "201163"   
[115] "83742"     "4838"      "79856"     "171586"    "112950"    "11093"    
[121] "84885"     "2630"      "10691"     "113802"    "6603"      "11212"    
[127] "100288142" "23498"     "23324"     "150465"    "30834"     "27238"    
[133] "55020"     "23276"     "64897"     "79411"     "4074"      "81929"    
[139] "9450"      "127396"    "91442"     "85403"     "8314"      "83700"    
[145] "23195"     "1200"      "9880"      "2021"      "55080"     "90522"    
[151] "3396"      "28511"     "1841"      "83606"     "51362"     "80346"    
[157] "90075"     "92305"     "9682"      "27252"     "51150"     "23759"    
[163] "9647"      "83746"     "23264"     "7461"      "5335"      "23345"    
[169] "10220"     "574407"    "53349"     "6609"      "79759"     "201255"   
[175] "5986"      "148268"    "126295"    "10201"     "51550"     "6996"     
[181] "22907"     "64393"     "253512"    "112752"    "55892"     "9681"     
[187] "84060"     "25821"     "4686"      "7189"      "8636"      "7694"     
[193] "284346"    "54621"     "403341"    "2135"      "23765"     "9659"     
[199] "125919"    "79791"     "54512"     "1537"      "84232"     "22928"    
[205] "81889"     "115286"    "137209"    "205428"    "84967"     "283989"   
[211] "81532"     "1436"      "79087"     "55039"     "10189"     "80213"    
[217] "162239"    "124751"    "51106"     "7565"      "84065"     "85002"    
[223] "11104"     "91582"     "79363"     "64403"     "643853"    "24144"    
[229] "215"       "54662"     "51669"     "282969"    "9672"      "163081"   
[235] "84503"     "81846"     "147929"    "30849"     "147923"    "8831"     
[241] "104"       "10174"     "84671"     "58491"     "6560"      "441478"   
[247] "51490"     "90196"     "57336"     "55937"     "3107"      "79833"    
[253] "91750"     "51409"     "54859"     "653440"    "22954"     "84911"    
[259] "4802"      "22828"     "1388"      "54978"     "91664"     "147040"   
[265] "157574"    "23053"     "100271715" "7511"      "1386"      "57570"    
[271] "5565"      "53354"     "150383"    "26517"     "25980"     "51058"    
[277] "11188"     "9374"      "10742"     "148413"    "7423"      "642938"   
[283] "23152"     "79646"     "6744"      "9777"      "55041"     "87"       
[289] "84892"     "5213"      "90133"     "23154"     "84256"     "9139"     
[295] "79918"     "22879"     "10404"     "22927"     "6426"      "79068"    
[301] "115704"    "3621"      "6483"      "57805"     "27185"     "222223"   
[307] "26130"     "17102

In [33]:
enrich[["ratio-cluster2 dwn"]][["GO"]]

,ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
GO:0036376,GO:0036376,sodium ion export across plasma membrane,1/2,12/13904,0.001725439,0.04589914,0.007247233,FXYD1,1
GO:1902307,GO:1902307,positive regulation of sodium ion transmembrane transport,1/2,18/13904,0.002587600,0.04589914,0.007247233,FXYD1,1
GO:0010765,GO:0010765,positive regulation of sodium ion transport,1/2,27/13904,0.003880143,0.04589914,0.007247233,FXYD1,1
GO:0140115,GO:0140115,export across plasma membrane,1/2,45/13904,0.006462715,0.04589914,0.007247233,FXYD1,1
GO:2000649,GO:2000649,regulation of sodium ion transmembrane transporter activity,1/2,45/13904,0.006462715,0.04589914,0.007247233,FXYD1,1
GO:1902305,GO:1902305,regulation of sodium ion transmembrane transport,1/2,52/13904,0.007466143,0.04589914,0.007247233,FXYD1,1
GO:1902476,GO:1902476,chloride transmembrane transport,1/2,61/13904,0.008755520,0.04589914,0.007247233,FXYD1,1
GO:0002028,GO:0002028,regulation of sodium ion transport,1/2,67/13904,0.009614639,0.04589914,0.007247233,FXYD1,1
GO:0006821,GO:0006821,chloride transport,1/2,74/13904,0.010616474,0.04589914,0.007247233,FXYD1,1


In [34]:
genes[["ratio-cluster0 dwn"]]

[1] "3682"      "3902"      "3594"      "55423"     "4773"      "80709"    
  [7] "843"       "921"       "478"       "55914"     "9744"      "841"      
 [13] "11184"     "9595"      "23157"     "79713"     "55133"     "5026"     
 [19] "10354"     "94240"     "10717"     "50640"     "9840"      "375248"   
 [25] "3394"      "9267"      "9295"      "11262"     "3561"      "54464"    
 [31] "9050"      "8669"      "84897"     "51177"     "55664"     "80067"    
 [37] "123720"    "9246"      "22806"     "3937"      "27334"     "9183"     
 [43] "23081"     "51699"     "1997"      "94101"     "83448"     "55180"    
 [49] "7403"      "134429"    "836"       "54468"     "80306"     "9318"     
 [55] "63901"     "114823"    "158160"    "147138"    "3718"      "128611"   
 [61] "8233"      "84250"     "10663"     "283234"    "165631"    "29909"    
 [67] "389289"    "147339"    "79415"     "80169"     "91523"     "1731"     
 [73] "283254"    "2529"      "5551"      "3669"      "154075"    "7318"     
 [79] "93349"     "159013"    "10320"     "151888"    "7535"      "339327"   
 [85] "5133"      "3821"      "10865"     "641"       "9447"      "60560"    
 [91] "57018"     "79753"     "126308"    "83759"     "79874"     "170960"   
 [97] "22821"     "11118"     "9867"      "4277"      "100506736" "3150"     
[103] "114836"    "28664"     "8631"      "22914"     "100528032" "4207"     
[109] "100271849" "644019"    "80830"     "65072"     "344697"    "53347"    
[115] "197135"    "645422"    "728139"    "5698"      "100271448" "3134"     
[121] "5533"      "387357"    "100852407" "6148"      "729558"    "10346"    
[127] "2749"      "5542"      "653247"    "3600"      "387751"    "399815"   
[133] "100421413" "80342"     "100422497" "645744"    "54883"     "54888"    
[139] "10734"     "23481"     "2997"      "7965"      "23125"     "9343"     
[145] "57150"     "63914"     "1716"      "550"       "54955"     "55234"    
[151] "79828"     "6627"      "6541"      "1737"      "64979"     "10201"    
[157] "163126"    "55929"     "124961"    "1616"      "113523636" "1375"     
[163] "5297"      "101929089" "11276"     "28755"     "112268198" "100033411"
[169] "8355"